In [1]:
import sys
sys.path.insert(0,'/global/homes/b/bpb/repos/metatlas/')
from metatlas import metatlas_objects as metob
from metatlas.compounds import structure_cleaning as metclean
# import sys
# sys.path.append('/global/project/projectdirs/openmsi/jupyterhub_libs/anaconda/lib/python2.7/site-packages')
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
import numpy as np
import pandas as pd

('NERSC=', True)


In [2]:
cpd_df = pd.read_csv('/project/projectdirs/metatlas/projects/unique_compounds.csv.gz')
print(cpd_df.shape)
cpd_df.head()

/global/common/software/m2650/python-cori/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,4,5,8,9,10,11,12,18,19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(200454, 22)


,chebi_id,chebi_url,description,formula,hmdb_id,hmdb_url,inchi,inchi_key,kegg_id,kegg_url,...,metacyc_id,mono_isotopic_molecular_weight,name,num_free_radicals,number_components,permanent_charge,pubchem_compound_id,pubchem_url,source,synonyms
0,CHEBI:73782,http://www.ebi.ac.uk/chebi/searchId.do?chebiId...,A member of the class of chalcones that is cha...,C23H24O5,NaN,NaN,InChI=1S/C23H24O5/c1-23(2)12-11-16-18(28-23)14...,CDFJITYOZNLONM-UYRXBGFRSA-N,NaN,NaN,...,NaN,380.162374,Praecanson A,0,1,0,9999819,http://pubchem.ncbi.nlm.nih.gov/compound/9999819,chebi///lipidmaps,Praecanson A///(Z)-Praecansone A///CHEBI:73782...
1,CHEBI:120014,http://www.ebi.ac.uk/chebi/searchId.do?chebiId...,NaN,C21H19N3OS,NaN,NaN,InChI=1S/C21H19N3OS/c1-13-8-9-16(10-14(13)2)22...,JZEFBZMLAJEBMC-UHFFFAOYSA-N,NaN,NaN,...,NaN,361.124883,AC1LKV71,0,1,0,999950,http://pubchem.ncbi.nlm.nih.gov/compound/999950,chebi,"AC1LKV71///MCULE-7620428541///2-(3,4-dimethyla..."
2,CHEBI:109792,http://www.ebi.ac.uk/chebi/searchId.do?chebiId...,NaN,C20H17ClN2O4,NaN,NaN,InChI=1S/C20H17ClN2O4/c1-25-15-8-6-14(7-9-15)1...,BQGDCUBBCXAHON-UHFFFAOYSA-N,NaN,NaN,...,NaN,384.087685,AC1LKV6I,0,1,0,999921,http://pubchem.ncbi.nlm.nih.gov/compound/999921,chebi,AC1LKV6I///HMS2348O14///MCULE-9139177920///5-[...
3,NaN,NaN,NaN,C23H47NO2,NaN,NaN,InChI=1S/C23H47NO2/c1-3-4-5-13-16-20(2)17-14-1...,MXSVVKSZJQKNBC-AVAPBHFXSA-N,NaN,NaN,...,NaN,369.360680,Penazetidine A,0,1,0,9999186,http://pubchem.ncbi.nlm.nih.gov/compound/9999186,lipidmaps,Penazetidine A///CHEMBL177780///2S-(hydroxymet...
4,CHEBI:29308,http://www.ebi.ac.uk/chebi/searchId.do?chebiId...,A dinitrile that is ethane substituted by two ...,C2N2,NaN,NaN,InChI=1S/C2N2/c3-1-2-4,JMANVNJQNLATNU-UHFFFAOYSA-N,NaN,NaN,...,NaN,52.006148,CYANOGEN,0,1,0,9999,http://pubchem.ncbi.nlm.nih.gov/compound/9999,chebi///wikidata,CYANOGEN///Ethanedinitrile///Oxalonitrile///Di...


In [3]:
df = pd.read_csv('/project/projectdirs/metatlas/projects/spectral_libraries/msmsrefs_nist20only.tab',sep='\t',usecols=['name','smiles','inchi_key','inchi','formula','exact_mass'])
print(df.shape[0])
df.drop_duplicates('inchi_key',inplace=True)
print(df.shape[0])
df.dropna(subset=['inchi_key'],inplace=True)
print(df.shape[0])
df = df[~df['inchi_key'].isin(cpd_df['inchi_key'])]
print(df.shape[0])
df.head()

1021694
27415
27414
17203


,name,smiles,inchi_key,inchi,formula,exact_mass
5,1-Palmitoyl-2-glutaryl phosphatidylcholine,CCCCCCCCCCCCCCCC(=O)OCC(COP(=O)(O)OCC[N+](C)(C...,CDZVJFRXJAUXPP-UHFFFAOYSA-O,InChI=1S/C29H56NO10P/c1-5-6-7-8-9-10-11-12-13-...,C29H57NO10P+,610.371460
6,N-(2-Chlorobenzyl)-2-(1H-indol-3-yl)ethanamine,ClC1=CC=CC=C1CNCCC1=CNC2=CC=CC=C12,ZSEPJTGBFYRCMJ-UHFFFAOYSA-N,InChI=1S/C17H17ClN2/c18-16-7-3-1-5-14(16)11-19...,C17H17ClN2,284.108026
94,"5-(2,3-Dichlorobenzyl)-1,3-thiazol-2-amine",N=C1NC=C(CC2=C(Cl)C(Cl)=CC=C2)S1,FHYYBLWDYNACNA-UHFFFAOYSA-N,InChI=1S/C10H8Cl2N2S/c11-8-3-1-2-6(9(8)12)4-7-...,C10H8Cl2N2S,257.978525
107,"2-Chloro-N-[5-(2-chlorobenzyl)-1,3-thiazol-2-y...",OC(CCl)=NC1=NC=C(CC2=CC=CC=C2Cl)S1,IFSQELGQAIQTRM-UHFFFAOYSA-N,InChI=1S/C12H10Cl2N2OS/c13-6-11(17)16-12-15-7-...,C12H10Cl2N2OS,299.989089
134,"N-(5-Benzyl-1,3-thiazol-2-yl)-2-[(4,6-dimethyl...",CC1=CC(C)=NC(SCC(O)=NC2=NC=C(CC3=CC=CC=C3)S2)=N1,LHMBJRUBDQYYSV-UHFFFAOYSA-N,InChI=1S/C18H18N4OS2/c1-12-8-13(2)21-18(20-12)...,C18H18N4OS2,370.092203


In [5]:
df.reset_index(drop=True,inplace=True)

In [6]:
def strip_non_ascii(string):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in string if 0 < ord(c) < 127)
    return ''.join(stripped)

# import time
# t0 = time.time()
# compound_list = []
counter = 0
for i,row in df.iterrows():
        compound = metob.Compound()
        compound.name = strip_non_ascii(row['name'])#, "utf-8",errors='ignore'))
    #     compound.description = unicode(df_row.Definition, "utf-8",errors='ignore')
    #     compound.synonyms = strip_non_ascii(df_row.synonyms)#, "utf-8",errors='ignore'))
        mol = Chem.MolFromInchi(row.inchi)
        
        formula = rdMolDescriptors.CalcMolFormula(mol)
        mw = Descriptors.ExactMolWt(mol)
        charge = Chem.GetFormalCharge(mol)
        free_radicals = Descriptors.NumRadicalElectrons(mol)

        compound.formula = formula
        compound.mono_isotopic_molecular_weight = mw
        compound.permanent_charge = charge
    #     try:
        compound.num_free_radicals = int(float(free_radicals))
    #     except:
    #         compound.num_free_radicals = int(float(max(df_row.num_radicals.split('///'))))
    # #     compound.number_components = int(1)
        ik = Chem.InchiToInchiKey(row.inchi)
        
        compound.inchi = row.inchi
        compound.inchi_key = ik
    #     compound.neutralized_2d_inchi = df_row.flat_inchi
    #     compound.neutralized_2d_inchi_key = df_row.flat_inchikey
    #     compound.neutralized_inchi = df_row.metatlas_inchi
    #     compound.neutralized_inchi_key = df_row.metatlas_inchi_key

        compound.source = 'nist20'#str(df_row.source_database)
    #     for url_index,database in enumerate(database_list):
    #         if getattr(df_row,database):
    #             setattr(compound, database, str(getattr(df_row,database)) )
    #             if url_list[url_index]:
    #                 ids = str(getattr(df_row,database)).split('///')
    #                 urls = []
    #                 for my_id in ids:
    #                     urls.append('%s%s'%(url_list[url_index],my_id))
    #                     if 'pubchem' in database:
    #                         setattr(compound, database.replace('_compound_id','_url'),'///'.join(urls) )
    #                     else:
    #                         setattr(compound, database.replace('_id','_url'),'///'.join(urls) )
#         compound_list.append(compound)
#         if ik!=row['inchi_key']:
#             print(i)
        metob.store(compound)
        counter += 1
        if counter == 100:
            print(i)
            counter = 0
# print time.time() - t0

/global/common/software/m2650/python-cori/lib/python2.7/site-packages/pymysql/cursors.py:166: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  result = self._query(query)


99
199
299
399
499
599
699
799
899
999
1099
1199
1299
1399
1499
1599
1699
1799
1899
1999
2099
2199
2299
2399
2499
2599
2699
2799
2899
2999
3099
3199
3299
3399
3499
3599
3699
5299
5399
5499
5599
5699
5799
5899
5999
6099
6199
6299
6399
6499
6599
6699
6799
6899
6999
7099
7199
7299
7399
7499
7599
7699
7799
7899
7999
8099
8199
8299
8399
8499
8599
8699
8799
8899
8999
9099
9199
9299
9399
9499
9599
9699
9799
9899
9999
10099
10199
10299
10399
10499
10599
10699
10799
10899
10999
11099
11199
11299
11399
11499
11599
11699
11799
11899
11999
12099
12199
12299
12399
12499
12599
12699
12799
12899
12999
13099
13199
13299
13399
13499
13599
13699
13799
13899
13999
14099
14199
14299
14399
14499
14599
14699
14799
14899
14999
15099
15199
15299
15399
15499
15599
15699
15799
15899
15999
16099
16199
16299
16399
16499
16599
16699
16799
16899
16999
17099
17199


In [ ]:
old_cpds = pd.read_csv('/project/projectdirs/metatlas/projects/unique_compounds.csv.gz')
old_cpds.head()

In [ ]:
# df = pd.read_pickle('/project/projectdirs/openmsi/projects/ben_run_pactolus/unique_compounds.pkl')
# df = pd.read_csv('/global/homes/b/bpb/Downloads/NonMetAtlas-ENZO.csv')
# df.rename(columns={'std_inchi_key':'metatlas_inchi_key',
#                   'std_inchi':'metatlas_inchi',
#                   'name':'common_name',
#                   'source':'source_database'},inplace=True)
cpds = [{'metatlas_inchi_key':'MUYNUOXIHSNESP-UHFFFAOYSA-N',
      'metatlas_inchi':'InChI=1S/C12H12O12/c13-3-1-19-11-5(3)21-7(15)9(11,17)23-10(18)8(16)22-6-4(14)2-20-12(6,10)24-11/h3-6,13-14,17-18H,1-2H2',
      'common_name':'Dehydroascorbic Acid Dimer',
      'source_database':'ChEBI'}]
df = pd.DataFrame(cpds)
df.tail()

In [ ]:
sum(df.metatlas_inchi_key.isin(old_cpds.inchi_key))

In [ ]:
def strip_non_ascii(string):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in string if 0 < ord(c) < 127)
    return ''.join(stripped)

import time
t0 = time.time()
# compound_list = []
counter = 0
for i,df_row in df.iterrows():
    if not df_row.metatlas_inchi_key in old_cpds.inchi_key:
        compound = metob.Compound()
        compound.name = strip_non_ascii(df_row.common_name)#, "utf-8",errors='ignore'))
    #     compound.description = unicode(df_row.Definition, "utf-8",errors='ignore')
    #     compound.synonyms = strip_non_ascii(df_row.synonyms)#, "utf-8",errors='ignore'))
        mol = Chem.MolFromInchi(df_row.metatlas_inchi)
        
        formula = rdMolDescriptors.CalcMolFormula(mol)
        mw = Descriptors.ExactMolWt(mol)
        charge = Chem.GetFormalCharge(mol)
        free_radicals = Descriptors.NumRadicalElectrons(mol)

        compound.formula = formula
        compound.mono_isotopic_molecular_weight = mw
        compound.permanent_charge = charge
    #     try:
        compound.num_free_radicals = int(float(free_radicals))
    #     except:
    #         compound.num_free_radicals = int(float(max(df_row.num_radicals.split('///'))))
    # #     compound.number_components = int(1)

        compound.inchi = df_row.metatlas_inchi
        compound.inchi_key = df_row.metatlas_inchi_key
    #     compound.neutralized_2d_inchi = df_row.flat_inchi
    #     compound.neutralized_2d_inchi_key = df_row.flat_inchikey
    #     compound.neutralized_inchi = df_row.metatlas_inchi
    #     compound.neutralized_inchi_key = df_row.metatlas_inchi_key

        compound.source = str(df_row.source_database)
    #     for url_index,database in enumerate(database_list):
    #         if getattr(df_row,database):
    #             setattr(compound, database, str(getattr(df_row,database)) )
    #             if url_list[url_index]:
    #                 ids = str(getattr(df_row,database)).split('///')
    #                 urls = []
    #                 for my_id in ids:
    #                     urls.append('%s%s'%(url_list[url_index],my_id))
    #                     if 'pubchem' in database:
    #                         setattr(compound, database.replace('_compound_id','_url'),'///'.join(urls) )
    #                     else:
    #                         setattr(compound, database.replace('_id','_url'),'///'.join(urls) )
    # #     compound_list.append(compound)
        metob.store(compound)
        counter += 1
        if counter == 100:
            print i
            counter = 0
print time.time() - t0

In [ ]:
compound

In [ ]:
df.shape

In [ ]:
database_list = ['chebi_id',
                 'hmdb_id',
                 'img_abc_id',
                 'kegg_id',
                 'lipidmaps_id',
                 'metacyc_id',
                 'pubchem_compound_id']
url_list = [
    'http://www.ebi.ac.uk/chebi/searchId.do?chebiId=',
    'http://www.hmdb.ca/metabolites/',
    '',
    'http://www.genome.jp/dbget-bin/www_bget?',
    'http://www.lipidmaps.org/data/LMSDRecord.php?LMID=',
    '',
    'http://pubchem.ncbi.nlm.nih.gov/compound/'
]

In [ ]:
def strip_non_ascii(string):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in string if 0 < ord(c) < 127)
    return ''.join(stripped)

import time
t0 = time.time()
# compound_list = []
counter = 0
for i,df_row in df.iterrows():
    compound = metob.Compound()
    compound.name = strip_non_ascii(df_row.common_name)#, "utf-8",errors='ignore'))
    compound.description = unicode(df_row.Definition, "utf-8",errors='ignore')
    compound.synonyms = strip_non_ascii(df_row.synonyms)#, "utf-8",errors='ignore'))
    
    compound.formula = str(df_row.formula)
    compound.mono_isotopic_molecular_weight = df_row.monoisotopic_mw
    compound.permanent_charge = int(float(df_row.charge))
    try:
        compound.num_free_radicals = int(float(df_row.num_radicals))
    except:
        compound.num_free_radicals = int(float(max(df_row.num_radicals.split('///'))))
    compound.number_components = int(1)
    
    compound.inchi = df_row.metatlas_inchi
    compound.inchi_key = df_row.metatlas_inchi_key
    compound.neutralized_2d_inchi = df_row.flat_inchi
    compound.neutralized_2d_inchi_key = df_row.flat_inchikey
    compound.neutralized_inchi = df_row.metatlas_inchi
    compound.neutralized_inchi_key = df_row.metatlas_inchi_key
    
    compound.source = str(df_row.source_database)
    for url_index,database in enumerate(database_list):
        if getattr(df_row,database):
            setattr(compound, database, str(getattr(df_row,database)) )
            if url_list[url_index]:
                ids = str(getattr(df_row,database)).split('///')
                urls = []
                for my_id in ids:
                    urls.append('%s%s'%(url_list[url_index],my_id))
                    if 'pubchem' in database:
                        setattr(compound, database.replace('_compound_id','_url'),'///'.join(urls) )
                    else:
                        setattr(compound, database.replace('_id','_url'),'///'.join(urls) )
#     compound_list.append(compound)
    metob.store(compound)
    counter += 1
    if counter == 1000:
        print i
        counter = 0
print time.time() - t0

In [ ]:
t0 = time.time()
for c in compound_list:
    metob.store(c)
print time.time() - t0

In [ ]:
14.003074*2+2*e

In [ ]:
# C8H8NO3
e = 0.000548579909
print 12.0*8 + 1.007825*8+14.003074+15.994915*3
print 12.0*8 + 1.007825*8+14.003074+15.994915*3 - e
print 166.049869536

In [ ]:
df[(df.charge==1) & (df.monoisotopoic_mw < 300)].head()

In [ ]:
df.tail(100)

In [ ]:
compound

In [ ]:
compound_list[0].reference_xrefs[0].name

In [ ]:
metatlas_compounds = metob.retrieve('Compounds',name = '%',username='*')

In [ ]:
print len(metatlas_compounds)

In [ ]:
metatlas_compounds[0]

In [ ]:
# metob.FunctionalSet()
# metob.to_dataframe(metatlas_compounds[:1])
mc1 = metob.Compound()
dbr = metob.ReferenceDatabase()
# mc1.reference_xrefs = [dbr,dbr]
print mc1()

In [ ]:
mc1()

In [ ]:
counter = 0
field_list = ['inchi','name']
metatlas_df = pd.DataFrame()
for compound in metatlas_compounds:
    for field in field_list:
        metatlas_df.loc[counter,field] = getattr(compound,field)
    counter += 1

In [ ]:
metatlas_df.head()

In [ ]:
def make_mol_from_smiles_and_inchi(row):
    mol= []
    if row['inchi']:
        mol = Chem.MolFromInchi(row['inchi'].encode('utf-8'))
    if mol:
        return mol

metatlas_df.ROMol = metatlas_df.apply(make_mol_from_smiles_and_inchi, axis=1)

In [ ]:
def desalt_compounds_in_dataframe(x):
    if x:
        if x.GetNumAtoms()>1:
            c = import_compounds.desalt(x)
            if c[1]:
                return c[0]
            else:
                return x
    
metatlas_df.ROMol = metatlas_df.ROMol.apply(desalt_compounds_in_dataframe)


In [ ]:
import_compounds = reload(import_compounds)
def neutralize_compounds_in_dataframe(x):
    if x:
        if x.GetNumAtoms()> 0:
            neutral_mol = []
            try:
                c = import_compounds.NeutraliseCharges(x)
                neutral_mol = c[0]
            except:
                pass
            if neutral_mol:
                return neutral_mol
            
metatlas_df.ROMol = metatlas_df.ROMol.apply(neutralize_compounds_in_dataframe)

In [ ]:
def calculate_num_radicals_in_dataframe(x):
    num_radicals = 0.0
    if x:
        num_radicals = Descriptors.NumRadicalElectrons(x)        
    return num_radicals

def calculate_formula_in_dataframe(x):
    formula = ''
    if x:
        formula = rdMolDescriptors.CalcMolFormula(x)
    return formula

def calculate_monoisotopic_mw_in_dataframe(x):
    mw = 0.0
    if x:
        mw = Descriptors.ExactMolWt(x)
    return mw

def calculate_inchi_in_dataframe(x):
    inchi = ''
    if x:
        try:
            inchi = Chem.MolToInchi(x)
        except:
            pass#This fails when can't kekulize mol
    return inchi

def calculate_flattened_inchi_in_dataframe(x):
    flattened_inchi = ''
    if x:
        sm = Chem.MolToSmiles(x).replace('@','')
        flattened_rdkit_mol = Chem.MolFromSmiles(sm)
        try:
            flattened_inchi = Chem.MolToInchi(flattened_rdkit_mol)
        except:
            pass#This fails when can't kekulize mol
    return flattened_inchi

def calculate_inchikey_in_dataframe(x):
    ik = ''
    if x:
        try:
            ik = Chem.InchiToInchiKey(x)
        except:
            pass#This fails when can't kekulize mol.  Carbo-cations are the culprit usually. 
    return ik

def calculate_charge_in_dataframe(x):
    if x:
        my_charge = Chem.GetFormalCharge(x)
        return my_charge
            
metatlas_df['charge'] = metatlas_df.ROMol.apply(calculate_charge_in_dataframe)
metatlas_df['formula'] = metatlas_df.ROMol.apply(calculate_formula_in_dataframe)
metatlas_df['monoisotopoic_mw'] = metatlas_df.ROMol.apply(calculate_monoisotopic_mw_in_dataframe)
metatlas_df['num_radicals'] = metatlas_df.ROMol.apply(calculate_num_radicals_in_dataframe)

metatlas_df['metatlas_inchi'] = metatlas_df.ROMol.apply(calculate_inchi_in_dataframe)
metatlas_df['metatlas_inchi_key'] = metatlas_df.metatlas_inchi.apply(calculate_inchikey_in_dataframe)

metatlas_df['flat_inchi'] = metatlas_df.ROMol.apply(calculate_flattened_inchi_in_dataframe)
metatlas_df['flat_inchikey'] = metatlas_df.flat_inchi.apply(calculate_inchikey_in_dataframe)


In [ ]:
metatlas_df = metatlas_df.where((pd.notnull(metatlas_df)), None)


In [ ]:
metatlas_df = metatlas_df[metatlas_df.metatlas_inchi_key != '']


In [ ]:
metatlas_df.to_pickle('backup of all metatlas compounds with flattened inchi and key.pkl')
metatlas_df.head()

In [ ]:
idx = metatlas_df.metatlas_inchi_key.isin(df.metatlas_inchi_key)

In [ ]:
not_found = metatlas_df[~idx]

In [ ]:
not_found = not_found.drop_duplicates()

In [ ]:
not_found.head(20)

In [ ]:
# not_found.to_csv('metatlas_compounds_not_found.csv')
# not_found.to_pickle('metatlas_compounds_not_found.pkl')

In [ ]:
not_found.shape